In [20]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
import os
from bs4 import BeautifulSoup 
import pandas as pd

In [21]:
driver_path = r'C:\Users\Pushkar Sharma\Desktop\Driver\chromedriver.exe'

img_path = 'FreepikImages'

if not os.path.exists(img_path):
    os.makedirs(img_path)

In [22]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
page_range = 1
extraction = {}

In [30]:
for i in range(1,page_range+1):
    
    url = f"https://www.freepik.com/search?ai=excluded&format=search&last_filter=people_ethnicity&last_value=indian&people=include&people_ethnicity=indian&query=kolkata+culture&type=photo&page={i}"
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(5)
    page_source = driver.page_source
    driver.quit()
    soup = BeautifulSoup(page_source, 'html.parser')
    fig_lst = soup.findAll('figure')
  
    for fig in fig_lst:
        img_link = fig.find('a', href=True)['href']
        get_title = requests.get(img_link, headers=headers)
        
        title_soup = BeautifulSoup(get_title.text, 'html.parser')
        title = title_soup.find('div', class_='detail__title overflow-lines').find('h1').get_text('strip=True')
        
        img_link = title_soup.find('img', class_='thumb').get('src')
        img_req = requests.get(img_link)
        
        img_name = title.replace(' ','_')
        
        f_name = os.path.join(img_path ,f'{img_name}.jpg')
        
        with open(f_name,'wb') as f:
            f.write(img_req.content)
            
        extraction[title] = img_link
        
        
   


In [25]:
df = pd.DataFrame(extraction.items(), columns=['Title', 'ImageLink'])

In [26]:
df.to_excel('DataExtraction.xlsx',index=False)